### Exercise 1 - Waze & Graph Theory Algorithms

### Question 1
1.1 Graph Creation

In [1]:
import sys
import osmnx as ox
import networkx as nx
from queue import PriorityQueue
import pandas as pd
import random 
import time as t
from typing import Tuple
from collections import Counter

In [2]:
class Graph(object):
    def __init__(self, num_vertices):
        self.vertices, self.graph = self.initialize(num_vertices)
        
    def initialize(self, num_vertices):
        # Initializes the vertices and the bare graph structure.
        # The graph is represented as a dictionary of dictionaries:
        # {from_vertex_i : {to_vertex_j : [non_traffic_time, traffic_time]}, ...}
        
        vertices = []
        for i in range(1,num_vertices):
            vertices.append(i)
        
        init_graph = {}
        for vertex in vertices:
            init_graph[vertex] = {}
            
        return vertices, init_graph

    def add_edge(self, start, end, cost, line, color, traffic, non_traffic):
        self.graph[start][end] = [cost/non_traffic, cost/traffic]

    def construct_graph(self):
        # Filling the graph with all information
        for vertex, edges in self.graph.items():
            for adjacent, value in edges.items():
                if self.graph[adjacent].get(vertex) == None:
                    self.graph[adjacent][vertex] = value

    def get_vertices(self):
        return self.vertices
    
    def get_outgoing_edges(self, from_vertex):
        # Returns a list of the verteices that have edges from 'from_vertex'
        edges = []
        for to_vertex in self.vertices:
            if self.graph[from_vertex].get(to_vertex) != None:
                edges.append(to_vertex)
        return edges
    
    def time(self, vertex1, vertex2, traffic=False):
        # Calculate the time to get from v1 to v2, depends on traffic
        return self.graph[vertex1][vertex2][0] if not traffic else self.graph[vertex1][vertex2][1]

In [3]:
# Constructing the bare graph with 26 vertices
g = Graph(27)

# A dictionary to store index for every place
idx_names = {
    1 : 'Post Office',
    2 : 'PT to 40',
    3 : 'Yarkon.',
    4 : 'Tikva',
    5 : 'Morasha',
    6 : '481 Road',
    7 : 'Geaa',
    8 : 'Em HaMoshavot',
    9 : 'Glilot',
    10 : 'KaKal',
    11 : 'Rokah',
    12 : 'Rokah-Ibn Gabirol',
    13 : 'Aluf Sade',
    14 : 'Hashalom-Halacha',
    15 : 'Rabin Square',
    16 : 'Glilot West',
    17 : 'Kohav Tzafon',
    18 : '471-40',
    19 : 'Moshe Dayan-Itzhak Sade',
    20 : 'Itzhak Rabin Mid',
    21 : 'Bar Ilan',
    22 : 'Shitrit-Rokah',
    23 : 'Dizengoff-Ibn Gabirol',
    24 : 'Kaplan',
    25 : 'Gat Rimon',
    26 : 'Neve Maoz'
}

# Adding the edges to the graph
g.add_edge(20, 13, cost=2.7, line=21, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(15, 23, cost=0.6, line=23, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(12, 15, cost=2.1, line=24, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(11, 17, cost=1.0, line=25, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(2, 3, cost=5.0, line=26, color='Blue', traffic=10, non_traffic=60)
g.add_edge(3, 4, cost=2.4, line=28, color='Blue', traffic=10, non_traffic=60)
g.add_edge(3, 5, cost=2.3, line=29, color='Blue', traffic=10, non_traffic=60)
g.add_edge(5, 8, cost=2.8, line=30, color='Blue', traffic=10, non_traffic=60)
g.add_edge(5, 9, cost=4.8, line=31, color='Blue', traffic=10, non_traffic=60)
g.add_edge(9, 10, cost=2.2, line=32, color='Blue', traffic=10, non_traffic=60)
g.add_edge(17, 12, cost=0.9, line=35, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(10, 11, cost=2.0, line=36, color='Blue', traffic=10, non_traffic=60)
g.add_edge(9, 16, cost=0.7, line=37, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(16, 17, cost=4.7, line=38, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(7, 6, cost=2.9, line=39, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(8, 7, cost=0.9, line=40, color='Blue', traffic=10, non_traffic=60)
g.add_edge(8, 7, cost=0.9, line=41, color='Blue', traffic=10, non_traffic=60)
g.add_edge(14, 7, cost=5.3, line=43, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(19, 20, cost=1.0, line=46, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(14, 20, cost=1.4, line=47, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(14, 19, cost=0.9, line=48, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(2, 18, cost=2.2, line=49, color='Blue', traffic=10, non_traffic=60)
g.add_edge(25, 18, cost=1.7, line=51, color='Blue', traffic=10, non_traffic=60)
g.add_edge(13, 21, cost=1.1, line=52, color='Blue', traffic=10, non_traffic=60)
g.add_edge(2, 6, cost=2.4, line=53, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(4, 6, cost=3.0, line=54, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(8, 22, cost=3.2, line=56, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(11, 22, cost=1.3, line=57, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(23, 24, cost=0.8, line=60, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(14, 24, cost=0.4, line=61, color='Purple', traffic=5, non_traffic=60)
g.add_edge(11, 14, cost=3.4, line=62, color='Purple', traffic=5, non_traffic=60)
g.add_edge(7, 21, cost=3.2, line=63, color='Blue', traffic=10, non_traffic=60)
g.add_edge(1, 18, cost=2.9, line=64, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(1, 25, cost=2.1, line=66, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(21, 25, cost=4.5, line=67, color='Blue', traffic=10, non_traffic=60)
g.add_edge(3, 26, cost=2.2, line=69, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(2, 26, cost=1.6, line=70, color='Yellow', traffic=20, non_traffic=30)
g.add_edge(1, 2, cost=1.7, line=71, color='Yellow', traffic=20, non_traffic=30)

# Building the full grpah
g.construct_graph()

In [4]:
def dijkstra(graph, start_vertex, target_vertex, traffic=False):
    unvisited_vertices = list(graph.get_vertices())   
    shortest_path = {}
    previous_vertices = {} 
    max_value = sys.maxsize
    
    # Init shortest path from vertex to any other with infinity
    for vertex in unvisited_vertices:
        shortest_path[vertex] = max_value
        
    # Init shortest path from start_vertex to itself with a 0
    shortest_path[start_vertex] = 0
    
    # Starting the algorithm
    while unvisited_vertices:
        current_min = None
        for vertex in unvisited_vertices:
            if current_min == None:
                current_min = vertex
            elif shortest_path[vertex] < shortest_path[current_min]:
                current_min = vertex

        # Updating the path to current_min's neighbors
        neighbors = graph.get_outgoing_edges(current_min)
        for neighbor in neighbors:
            time = shortest_path[current_min] + graph.time(current_min, neighbor, traffic)
            if time < shortest_path[neighbor]:
                shortest_path[neighbor] = time
                previous_vertices[neighbor] = current_min
 
        # Removing current_min vertex from the list
        unvisited_vertices.remove(current_min)
    
    path = []
    vertex = target_vertex
    while vertex != start_vertex:
        path.append(vertex)
        vertex = previous_vertices[vertex]
    path.append(start_vertex)
    
    return list(reversed(path)), round(shortest_path[target_vertex]*60 ,2)

1.2 Finding Shortest Path from Petah Tikva's Post Office to Rabin Square - Without traffic

In [5]:
shortest_path, time = dijkstra(graph=g, start_vertex=1, target_vertex=15, traffic=False)
time = '\033[1m' + str(time) + '\033[0m'
print(f'The shortest path to drive from the Post Office to Rabin Square without traffic takes {time} minutes as follows:\n')
for i, idx in enumerate(shortest_path):
    if i < len(shortest_path) - 1:
        print(f'{idx_names[idx]} ->',end=' ')
    else:
        print(idx_names[idx],'\n')

The shortest path to drive from the Post Office to Rabin Square without traffic takes 21.2 minutes as follows:

Post Office -> Gat Rimon -> Bar Ilan -> Aluf Sade -> Itzhak Rabin Mid -> Hashalom-Halacha -> Kaplan -> Dizengoff-Ibn Gabirol -> Rabin Square 



1.3 Finding Shortest Path from Petah Tikva's Post Office to Rabin Square - With traffic

In [6]:
shortest_path, time = dijkstra(graph=g, start_vertex=1, target_vertex=15, traffic=True)
time = '\033[1m' + str(time) + '\033[0m'
print(f'The shortest path to drive from the Post Office to Rabin Square with traffic takes {time} minutes as follows:\n')
for i, idx in enumerate(shortest_path):
    if i < len(shortest_path) - 1:
        print(f'{idx_names[idx]} ->',end=' ')
    else:
        print(idx_names[idx],'\n')

The shortest path to drive from the Post Office to Rabin Square with traffic takes 45.9 minutes as follows:

Post Office -> PT to 40 -> 481 Road -> Geaa -> Hashalom-Halacha -> Kaplan -> Dizengoff-Ibn Gabirol -> Rabin Square 



1.4 Finding Shortest Path from Petah Tikva's Post Office to Rabin Square - With traffic & blocked road

In [7]:
# Changing PT to 40 -> 481 Road speed to be almost 0 == blocking the road
g.add_edge(2, 6, cost=2.4, line=53, color='Yellow', traffic=0.00000001, non_traffic=0.00000001)

shortest_path, time = dijkstra(graph=g, start_vertex=1, target_vertex=15, traffic=True)
time = '\033[1m' + str(time) + '\033[0m'
print(f'The shortest path to drive from the Post Office to Rabin Square with traffic takes {time} minutes as follows:\n')
for i, idx in enumerate(shortest_path):
    if i < len(shortest_path) - 1:
        print(f'{idx_names[idx]} ->',end=' ')
    else:
        print(idx_names[idx],'\n')

The shortest path to drive from the Post Office to Rabin Square with traffic takes 61.2 minutes as follows:

Post Office -> Gat Rimon -> Bar Ilan -> Aluf Sade -> Itzhak Rabin Mid -> Hashalom-Halacha -> Kaplan -> Dizengoff-Ibn Gabirol -> Rabin Square 



###  Question 2

2.1 Editing dijkstra0 to show the number of visited nodes

In [8]:
ox.settings.log_console=True
ox.settings.use_cache=True
mode = 'drive'        # 'drive', 'bike', 'walk'
graph = ox.graph_from_place('Tel Aviv, Israel', network_type = mode)

start_lnglat = (34.794, 32.070)
end_lnglat = (34.802, 32.115)

orig_node = ox.nearest_nodes(graph, *start_lnglat)
dest_node = ox.nearest_nodes(graph, *end_lnglat)

shortest_route = nx.shortest_path(graph, orig_node, dest_node, weight='time')

In [9]:
def dijkstra0(graph, start_node, end_node):
    pq = PriorityQueue()
    pq.put((0, [start_node]))
    
    visited = set()
    while not pq.empty():
        (dist, route) = pq.get()
        visited.add(route[-1])
        
        if route[-1] == end_node:
            print('Found Route!')
            print(f'The number of visited nodes in that route is: {len(visited)}') # Printing the number of nodes
            return route
        
        for from_node, to_node, att in graph.edges(route[-1], data=True):
            if to_node not in visited:
                pq.put((dist + att['length'], route + [to_node]))
                
    print('Route not found!')
    
my_route = dijkstra0(graph, orig_node, dest_node)

Found Route!
The number of visited nodes in that route is: 4794


2.2 Showing the length of the shortest path

In [10]:
dist = 0
for i in range(len(my_route)-1):
    for u, v, length in graph.edges(my_route[i], data='length'):
        if u == my_route[i] and v == my_route[i+1]:
            dist += length
print(f'The length of the shortest path is: {round(dist,2)}')

The length of the shortest path is: 5854.31


2.3 Printing the chosen shortest path

In [11]:
count = 0
for i in range(len(my_route)-1):
    for u, v, data in graph.edges(my_route[i], data=True):
        if u == my_route[i] and v == my_route[i+1]:
            count += data['length']
            name = data['name']
            weight = data['length']
            print(f'Node ID: {u}, Road name: {name}, Weight: {round(weight,2)}, Total length: {round(count,2)}')

Node ID: 442893209, Road name: יגאל אלון, Weight: 147.0, Total length: 147.0
Node ID: 442893216, Road name: יגאל אלון, Weight: 83.92, Total length: 230.92
Node ID: 442893205, Road name: יגאל אלון, Weight: 15.54, Total length: 246.45
Node ID: 2473987340, Road name: יגאל אלון, Weight: 26.93, Total length: 273.39
Node ID: 2137606211, Road name: יגאל אלון, Weight: 102.21, Total length: 375.59
Node ID: 8914420752, Road name: יגאל אלון, Weight: 7.97, Total length: 383.56
Node ID: 8914420754, Road name: יגאל אלון, Weight: 189.54, Total length: 573.1
Node ID: 2473987367, Road name: יגאל אלון, Weight: 8.38, Total length: 581.48
Node ID: 7284338856, Road name: נחלת יצחק, Weight: 7.62, Total length: 589.1
Node ID: 2344604154, Road name: מוזס, Weight: 98.38, Total length: 687.48
Node ID: 1411567099, Road name: מוזס, Weight: 135.42, Total length: 822.9
Node ID: 3762374460, Road name: מוזס, Weight: 144.21, Total length: 967.1
Node ID: 10018819914, Road name: מנחם בגין, Weight: 48.84, Total length: 1

In [12]:
# This is the same as above with Pandas table
# lengths = []
# names = []
# count = [0]
# for i in range(len(my_route)-1):
#     for u, v, data in graph.edges(my_route[i], data=True):
#         if u == my_route[i] and v == my_route[i+1]:
#             count.append(count[-1] + data['length'])
#             names.append(data['name'])
#             lengths.append(data['length'])
            
# data = {'Node ID': my_route[:-1],
#         'Rode Name': names,
#         'Rode Length': lengths,
#         'Total Length': count[1:]}
# df = pd.DataFrame(data, index=range(1,len(my_route)))
# df

2.4 Writing dijkstra1, and printing the results as before

In [13]:
def dijkstra1(graph, start_node, end_node):
    pq = PriorityQueue()
    pq.put((0, [start_node]))
    visited = set()
    
    while not pq.empty():
        (dist, route) = pq.get()
        visited.add(route[-1])
        
        if route[-1] == end_node:
            print('Found Route!')
            print(f'The number of visited nodes in that route is: {len(visited)}') # Printing the number of nodes
            return route
        for from_node, to_node, att in graph.edges(route[-1], data=True):
            if to_node == end_node:
                print('Found Route!')
                print(f'The number of visited nodes in that route is: {len(visited)}') # Printing the number of nodes
                return route + [to_node]
            if to_node not in visited:
                pq.put((dist + att['length'], route + [to_node]))
                
    print('Route not found!')
    
my_route1 = dijkstra1(graph, orig_node, dest_node)

Found Route!
The number of visited nodes in that route is: 4686


In [14]:
dist1 = 0
for i in range(len(my_route1)-1):
    for u, v, length in graph.edges(my_route1[i], data='length'):
        if u == my_route[i] and v == my_route1[i+1]:
            dist1 += length
print(f'The length of the shortest path is: {round(dist1,2)}')

The length of the shortest path is: 5854.31


In [15]:
count = 0
for i in range(len(my_route1)-1):
    for u, v, data in graph.edges(my_route1[i], data=True):
        if u == my_route1[i] and v == my_route1[i+1]:
            count += data['length']
            name = data['name']
            weight = data['length']
            print(f'Node ID: {u}, Road name: {name}, Weight: {round(weight,2)}, Total length: {round(count,2)}')

Node ID: 442893209, Road name: יגאל אלון, Weight: 147.0, Total length: 147.0
Node ID: 442893216, Road name: יגאל אלון, Weight: 83.92, Total length: 230.92
Node ID: 442893205, Road name: יגאל אלון, Weight: 15.54, Total length: 246.45
Node ID: 2473987340, Road name: יגאל אלון, Weight: 26.93, Total length: 273.39
Node ID: 2137606211, Road name: יגאל אלון, Weight: 102.21, Total length: 375.59
Node ID: 8914420752, Road name: יגאל אלון, Weight: 7.97, Total length: 383.56
Node ID: 8914420754, Road name: יגאל אלון, Weight: 189.54, Total length: 573.1
Node ID: 2473987367, Road name: יגאל אלון, Weight: 8.38, Total length: 581.48
Node ID: 7284338856, Road name: נחלת יצחק, Weight: 7.62, Total length: 589.1
Node ID: 2344604154, Road name: מוזס, Weight: 98.38, Total length: 687.48
Node ID: 1411567099, Road name: מוזס, Weight: 135.42, Total length: 822.9
Node ID: 3762374460, Road name: מוזס, Weight: 144.21, Total length: 967.1
Node ID: 10018819914, Road name: מנחם בגין, Weight: 48.84, Total length: 1

In [16]:
# This is the same as above with Pandas table
# lengths = []
# names = []
# count = [0]
# for i in range(len(my_route1)-1):
#     for u, v, data in graph.edges(my_route1[i], data=True):
#         if u == my_route1[i] and v == my_route1[i+1]:
#             count.append(count[-1] + data['length'])
#             names.append(data['name'])
#             lengths.append(data['length'])
            
# data = {'Node ID': my_route1[:-1],
#         'Rode Name': names,
#         'Rode Length': lengths,
#         'Total Length': count[1:]}
# df = pd.DataFrame(data, index=range(1,len(my_route1)))
# df

2.5 Comparing between dijkstra0 to dijkstra1

In [17]:
# Non-printing dijkstra0 & dijkstra1
def dijkstra0(graph, start_node, end_node):
    pq = PriorityQueue()
    pq.put((0, [start_node]))
    visited = set()
    while not pq.empty():
        (dist, route) = pq.get()
        visited.add(route[-1])
        if route[-1] == end_node:
            return route
        for from_node, to_node, att in graph.edges(route[-1], data=True):
            if to_node not in visited:
                pq.put((dist + att['length'], route + [to_node]))
                
def dijkstra1(graph, start_node, end_node):
    pq = PriorityQueue()
    pq.put((0, [start_node]))
    visited = set()
    while not pq.empty():
        (dist, route) = pq.get()
        visited.add(route[-1])
        if route[-1] == end_node:
            return route
        for from_node, to_node, att in graph.edges(route[-1], data=True):
            if to_node == end_node:
                return route + [to_node]
            if to_node not in visited:
                pq.put((dist + att['length'], route + [to_node]))

In [18]:
# Initializing values
# Ramat Aviv, Tel Aviv
max_lat = 32.108110
max_lng = 34.796760

# Florentin, Tel Aviv
min_lat = 32.057710
min_lng = 34.770130

d0 = 0
d1 = 0

N = 250
num_of_iter = N

graph = ox.graph_from_place('Tel Aviv, Israel', network_type = mode)

In [19]:
start = t.time()
for i in range(0,N):
    if i % 50 == 0:
        print(f'Iteration: {i}')
    graph0 = graph.copy()
    random_lat_start = random.uniform(min_lat,max_lat)
    random_lng_start = random.uniform(min_lng,max_lng)
    random_lat_end = random.uniform(min_lat,max_lat)
    random_lng_end = random.uniform(min_lng,max_lng)
    
    start_lnglat = (random_lng_start, random_lat_start)
    end_lnglat =  (random_lng_end, random_lat_end)
    orig_node = ox.nearest_nodes(graph0, *start_lnglat)
    dest_node = ox.nearest_nodes(graph0, *end_lnglat)
    
    shortest_route0 = dijkstra0(graph0, orig_node, dest_node)
    shortest_route1 = dijkstra1(graph0, orig_node, dest_node)
    
    if shortest_route0 and shortest_route1:
        for i in range(len(shortest_route0)-1):
            for u, v, length in graph.edges(shortest_route0[i], data='length'):
                if u == shortest_route0[i] and v == shortest_route0[i+1]:
                    d0 += length

        for i in range(len(shortest_route1)-1):
            for u, v, length in graph.edges(shortest_route1[i], data='length'):
                if u == shortest_route1[i] and v == shortest_route1[i+1]:
                    d1 += length
    else:
        num_of_iter -= 1
end = t.time()
print(f'Elapsed time: {end - start}')
d0_avg = '\033[1m' + str(round(d0/num_of_iter,2)) + '\033[0m'
d1_avg = '\033[1m' + str(round(d1/num_of_iter,2)) + '\033[0m'

Iteration: 0
Iteration: 50
Iteration: 100
Iteration: 150
Iteration: 200
Elapsed time: 123.80285501480103


In [20]:
print(f'The average length of {N} shortest paths with dijkstra0 is: {d0_avg}')
print(f'The average length of {N} shortest paths with dijkstra1 is: {d1_avg}\n')
print('The results are very close, but not identical.\nWe can see that on average dijkstra0 finds better (shorter) routes!\n')
print('dijkstra1 stops as soon as it reaches the end_node for the first time, while dijkstra0 does not.')
print('Thus, dijkstra1 might miss better routes when trying to find the shortest path.\n')
print('When dijkstra0 reaches the end_node for the first time, it puts the end_node back to the PriorityQueue.')
print('Now, it can explore more possible routes to minimize the shortest path!')

The average length of 250 shortest paths with dijkstra0 is: 3009.82
The average length of 250 shortest paths with dijkstra1 is: 3014.39

The results are very close, but not identical.
We can see that on average dijkstra0 finds better (shorter) routes!

dijkstra1 stops as soon as it reaches the end_node for the first time, while dijkstra0 does not.
Thus, dijkstra1 might miss better routes when trying to find the shortest path.

When dijkstra0 reaches the end_node for the first time, it puts the end_node back to the PriorityQueue.
Now, it can explore more possible routes to minimize the shortest path!


### Question 3

a. It is hard to calculate the actual ETA that way, because Waze bases its calculations on the same initial map for the whole journy with the same<br>traffics and speeds according to that map only, and it does not take into account traffics that may occur X minutes after the journey has started.

b. It is possible to give a better ETA by cutting the route to X chunks based on its length, and calculate the ETA from one chunk to the other based on the first traffic map according to the specific starting time of that chunk.<br>
Thus, we can "3D map" the whole journey, and if I left my home at 8:00 for example, Waze will calculate the time from my house to the first chunk based on that traffic map, and from there, assuming I should arrive there at 8:10, Waze will calculate the next chunk based on the known statistics of 8:10.
In the end, it calculates all the routes and merging them toghether for one complete ETA.

### Question 4

In [21]:
class TrieNode(object):
    """
    Our trie node implementation. Very basic. but does the job
    """
    
    def __init__(self, char: str):
        self.char = char
        self.children = []
        # Is it the last character of the word.
        self.word_finished = False
        # How many times this character appeared in the addition process
        self.counter = 1

def read_text(file):
    f = open(file, 'r', encoding='utf-8')
    text0 = f.read()
    f.close()
    
    text0 = text0.lower()
    
    legalABC = 'abcdefghijklmnopqrstuvwxyz '
    text = ''
    last_was_space = True
    
    for ch in text0:
        if ch == '\n':
            ch = ' '
        if ch in legalABC:
            if ch != " ":
                text = text + ch
                last_was_space = False
            if ch == " " and last_was_space == False:
                text= text + ch
                last_was_space = True
    return text

def add(root, word: str):
    """
    Adding a word in the trie structure
    """
    node = root
    for char in word:
        found_in_child = False
        # Search for the character in the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found it, increase the counter by 1 to keep track that another word has it as well
                child.counter += 1
                # And point the node to the child that contains this char
                node = child
                found_in_child = True
                break
        # We did not find it so add a new chlid
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            # And then point node to the new child
            node = new_node
    # Everything finished. Mark it as the end of a word.
    node.word_finished = True

def find_prefix(root, prefix: str) -> Tuple[bool, int, TrieNode]:
    """
    Check and return 
      1. If the prefix exsists in any of the words we added so far
      2. If yes then how many words actually have the prefix
      3. The node where the profix ends in the Trie structure
    """
    node = root
    # If the root node has no children, then return False.
    # Because it means we are trying to search in an empty trie
    if not root.children:
        return False, 0, node
    for char in prefix:
        char_not_found = True
        # Search through all the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found the char existing in the child.
                char_not_found = False
                # Assign node as the child containing the char and break
                node = child
                break
        # Return False anyway when we did not find a char.
        if char_not_found:
            return False, 0, node
    # Well, we are here means we have found the prefix. Return true to indicate that
    # And also the counter of the last node. This indicates how many words have this
    # prefix
    return True, node.counter, node


def list_all_children(root, prefix: str):
    """
    Returns all the children of the node at the end of a profix, with the counter for each child
    """
    node = root
    auto_complete_word = prefix
    (exists, counter, node) = find_prefix(root, auto_complete_word)
    print('This prefix occours ',counter, ' times.')
    
    for child in node.children:
        print(child.char, child.counter)
    
    return

def auto_complete(root, prefix: str) -> str:
    """
    This function is *not* written yet. You will complete it in the homework assignment 
    Suggests the most probable ending to the prefix typed so far by the user
    """ 
    node = root
    prefix = prefix.lower()
    auto_complete_word = prefix

    # Here *YOU* will write the code to suggest most probable word as auto-complete
    exists, counter, node = find_prefix(root, auto_complete_word)
    if node.word_finished:
        return auto_complete_word
    
    if exists == False:
        return prefix

    finish = False
    cur_word = prefix
    
    while not finish:
        max_prob = 0
        cur_node = None
        sum_prob = 0
        
        for child in node.children:
            sum_prob += child.counter

        for child in node.children:
            prob = child.counter/sum_prob
            if prob > max_prob:
                cur_node = child
                max_prob = prob
        
        node = cur_node
        
        if node.word_finished:
            finish = True
            
        cur_word += node.char
         
    return cur_word

def init_auto_complete(train_text):
    root = TrieNode('*')
    working_text = read_text(train_text)
    while working_text.find(' ') != -1 or working_text.find('\n') != -1:  # as long as there are more words in the text
        end_word_index = working_text.find(' ')
        current_word = working_text[0:end_word_index]
        add(root, current_word)
        working_text = working_text[end_word_index+1:]
        
    user_word = input("Enter a word: ")
    user_text_so_far = ''
    for i in user_word:
        user_text_so_far = user_text_so_far + i
        suggested = auto_complete(root, user_text_so_far)
        print(f'User Text: {user_text_so_far}\nAuto Complete Suggests: {suggested}\n')

In [22]:
init_auto_complete('text-EN.txt')

Enter a word: why
User Text: w
Auto Complete Suggests: which

User Text: wh
Auto Complete Suggests: which

User Text: why
Auto Complete Suggests: why



### Question 5

In [23]:
text = read_text('text-EN.txt')      

In [24]:
def freq_table(text):
    ht = {}
    for char in text:
        if char not in ht:
            ht[char] = 1
        else:
            ht[char] += 1
    ht = {k: v for k, v in sorted(ht.items(), key=lambda item: item[1], reverse=True)}   
    return ht

5.1 Frequency table as an array (python list):

In [25]:
freq_list = sorted(list(freq_table(text).items()))
freq_list

[(' ', 7154),
 ('a', 2820),
 ('b', 550),
 ('c', 1141),
 ('d', 1214),
 ('e', 4234),
 ('f', 794),
 ('g', 738),
 ('h', 1729),
 ('i', 2546),
 ('j', 37),
 ('k', 171),
 ('l', 1506),
 ('m', 819),
 ('n', 2569),
 ('o', 2632),
 ('p', 630),
 ('q', 26),
 ('r', 2194),
 ('s', 2155),
 ('t', 3340),
 ('u', 969),
 ('v', 422),
 ('w', 629),
 ('x', 50),
 ('y', 648),
 ('z', 16)]

5.2 The Huffman Code:

In [26]:
class NodeTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return self.left, self.right

    def __str__(self):
        return self.left, self.right

def huffman_code_tree(node, binString=''):
    '''
    Function to find Huffman Code
    '''
    if type(node) is str:
        return {node: binString}
    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, binString + '0'))
    d.update(huffman_code_tree(r, binString + '1'))
    return d

def make_tree(nodes):
    '''
    Function to make tree
    :param nodes: Nodes
    :return: Root of the tree
    '''
    while len(nodes) > 1:
        (key1, c1) = nodes[-1]
        (key2, c2) = nodes[-2]
        nodes = nodes[:-2]
        node = NodeTree(key1, key2)
        nodes.append((node, c1 + c2))
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
    return nodes[0][0]

def init_huffman(text, print_code=False):
    string = text
    freq = freq_table(text)
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    node = make_tree(freq)
    encoding = huffman_code_tree(node)
    if print_code:
        for i in encoding:
            print(f'{i} : {encoding[i]}')
    return encoding, freq

5.3 Encoding the first 1000 letters of our text file:

In [27]:
first_encoding, first_freq_table = init_huffman(text, True)

h : 0000
u : 00010
c : 00011
e : 001
s : 0100
r : 0101
b : 011000
w : 011001
d : 01101
i : 0111
n : 1000
o : 1001
p : 101000
y : 101001
x : 101010000
j : 1010100010
z : 10101000110
q : 10101000111
k : 10101001
v : 1010101
g : 101011
a : 1011
l : 11000
f : 110010
m : 110011
t : 1101
  : 111


In [28]:
hf_text = read_text('book-hw1.txt')
hf_text = hf_text[0:1000]

encoding_sum = 0
for char in hf_text:
    encoding_sum += len(first_encoding[char])
storage_saved = round(100-encoding_sum/80, 2)
print(f'By using Huffman coding, we only need {encoding_sum} bits, instead of 8000 bits!')
print(f'Relative efficiency: we save {storage_saved}% of storage by using the Huffman coding!')

By using Huffman coding, we only need 4167 bits, instead of 8000 bits!
Relative efficiency: we save 47.91% of storage by using the Huffman coding!


5.4 Using another text file

In [29]:
# We chose the book: The Secret Adversary by Agatha Christie
new_text = read_text('agatha.txt')

In [30]:
second_encoding, second_freq_table = init_huffman(new_text, True)
second_freq_table

r : 0000
h : 0001
s : 0010
m : 00110
x : 001110000
z : 0011100010
q : 0011100011
j : 00111001
v : 0011101
f : 001111
e : 010
n : 0110
i : 0111
u : 10000
p : 100010
g : 100011
o : 1001
a : 1010
l : 10110
d : 10111
c : 110000
w : 110001
k : 1100100
b : 1100101
y : 110011
t : 1101
  : 111


[(' ', 75059),
 ('e', 40586),
 ('t', 30507),
 ('a', 24585),
 ('o', 24464),
 ('i', 21639),
 ('n', 21300),
 ('s', 19705),
 ('h', 19689),
 ('r', 17870),
 ('d', 14106),
 ('l', 13092),
 ('u', 11349),
 ('m', 9519),
 ('y', 7840),
 ('w', 7659),
 ('c', 7304),
 ('g', 6704),
 ('p', 6280),
 ('f', 6178),
 ('b', 4720),
 ('k', 2953),
 ('v', 2866),
 ('j', 956),
 ('x', 388),
 ('q', 339),
 ('z', 215)]

In [31]:
df0 = pd.DataFrame(first_freq_table)
df1 = pd.DataFrame(second_freq_table)
df = pd.concat([df0, df1], axis=1)

df.columns = ['Letter', 'Frequency', 'Letter', 'Frequency']
columns=[('First Table:','Letter'),('First Table:','Frequency'),('Second Table:','Letter'),('Second Table:','Frequency')]
df.columns=pd.MultiIndex.from_tuples(columns)

df

First Table:           Second Table:          
         Letter Frequency        Letter Frequency
0                    7154                   75059
1             e      4234             e     40586
2             t      3340             t     30507
3             a      2820             a     24585
4             o      2632             o     24464
5             n      2569             i     21639
6             i      2546             n     21300
7             r      2194             s     19705
8             s      2155             h     19689
9             h      1729             r     17870
10            l      1506             d     14106
11            d      1214             l     13092
12            c      1141             u     11349
13            u       969             m      9519
14            m       819             y      7840
15            f       794             w      7659
16            g       738             c      7304
17            y       648             g      6704
18            p       630             p      6280
19            w       629             f      6178
20            b       550             b      4720
21            v       422             k      2953
22            k       171             v      2866
23            x        50             j       956
24            j        37             x       388
25            q        26             q       339
26            z        16             z       215

We can see a difference between the first and the second table.<br>
On both tables, the most common letters are '$~~$', 'e', 't', 'a', and 'o'.<br>
After that, the order of the letters is different, which changes the encoding of those letters.<br>
Changing the encoding, might change the number of bits that is required to store the most frequent letters.<br>
By doing so, the relative efficiency can be better or worst, depends on the letters frequenct of the text.

In [32]:
# However, the change is not that signifficant:
hf_text = read_text('book-hw1.txt')
hf_text = hf_text[0:1000]

sec_encoding_sum = 0
for char in hf_text:
    sec_encoding_sum += len(second_encoding[char])
sec_storage_saved = round(100-sec_encoding_sum/80,2)
print(f'By using Huffman coding, we only need {sec_encoding_sum} bits, instead of 8000 bits!')
print(f'Relative efficiency: we save {sec_storage_saved}% of storage by using the Huffman coding!\n')
print(f'With the first text we used:  {encoding_sum} bits to store the information.')
print(f'With the second text we used: {sec_encoding_sum} bits to store the information.\n')
print(f'The difference is only about {round(sec_storage_saved-storage_saved,2)}% of storage saving.')

By using Huffman coding, we only need 4159 bits, instead of 8000 bits!
Relative efficiency: we save 48.01% of storage by using the Huffman coding!

With the first text we used:  4167 bits to store the information.
With the second text we used: 4159 bits to store the information.

The difference is only about 0.1% of storage saving.
